In [ ]:
import os
code_path = os.getcwd()
data_path = code_path + 'Original X-ray Image Dataset'
covid_path = data_path + '/covid/'
none_path = data_path + '/none/'
pneumonia_path = data_path + '/pneumonia/'
print(len(os.listdir(covid_path))) #125
print(len(os.listdir(none_path))) #500
print(len(os.listdir(pneumonia_path))) #500

In [ ]:
from tensorflow.keras.models import Sequential, load_model, Model
import matplotlib.image as mpimg
from keras_preprocessing.image import img_to_array
import PIL
import numpy as np
import keras.preprocessing.image
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

In [ ]:
def makemodel(k):
      global model
      K.clear_session()
      model_input = layers.Input(shape = (256,256,3))
      x = layers.Conv2D(8, 3, 1, padding='same', use_bias=False)(model_input)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(16, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(32, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(16, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(32, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(64, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(32, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(64, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(128, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(64, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(128, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.MaxPooling2D((2,2),strides=2)(x)
      x = layers.Conv2D(256, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(128, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(256, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.ZeroPadding2D(((2,0),(2,0)))(x)
      x = layers.Conv2D(128, 1, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(256, 3, 1, padding='same', use_bias=False)(x)
      x = layers.BatchNormalization()(x)
      x = layers.LeakyReLU(alpha=0.1)(x)
      x = layers.Conv2D(k, k, 1, padding="same")(x)
      x = layers.ReLU()(x)
      x = layers.BatchNormalization()(x)
      x_ls = layers.Flatten()(x)
      
      "orthogonal constraint"
      ls_split_dim = k
      x_ls_norm = layers.Lambda(lambda t: K.l2_normalize(t, axis=1))(x_ls)
      ls_split = layers.Lambda(lambda tensor: tf.split(tensor, [int(K.int_shape(x_ls)[1]/ls_split_dim)]*ls_split_dim,axis=1))(x_ls_norm)
      ls_stack = layers.Lambda(lambda tensor: tf.stack(tensor, axis=2))(ls_split)
      ls_utu = layers.Lambda(lambda tensor: tf.linalg.matmul(tensor,tensor,transpose_a=True))(ls_stack)
      ls_utu_diag = layers.Lambda(lambda tensor: tf.compat.v1.matrix_diag_part(tensor)-1)(ls_utu)
      ls_utu2 = layers.Lambda(lambda tensor: tf.compat.v1.matrix_set_diag(tensor,ls_utu_diag))(ls_utu)
      l_orths = layers.Lambda(lambda tensor: K.sqrt(K.sum(K.square(tensor),axis=[1,2])))(ls_utu2)
      l_orths = layers.Reshape((1,))(l_orths)
      model_output = layers.Dense(3, activation="softmax")(x_ls)
      model = Model(model_input, [model_output, l_orths])
      return model

In [ ]:
def image_preprocessing(image_path):
  image_loaded=tf.python.keras.preprocessing.image.load_img(image_path, color_mode='rgb', 
  target_size= (256,256))
  image_loaded=np.asarray(image_loaded)
  image = image_loaded.astype('float32')/255
  image = img_to_array(image)
  x = np.expand_dims(image, axis=0)
  image_class = np.argmax(image_class_vector)

In [ ]:
def get_activation_map(image_path, image_class_vector, img_size):        
        image_preprocessing(image_path)
        makemodel(k)

        model.load_weights(dataset_path+'OS-model-weights.h5')
        class_weights = model.layers[-2].get_weights()[0]
        final_conv_layer = model.layers[-12]

        get_output = tf.keras.backend.function([model.layers[0].input], 
                                               [final_conv_layer.output, model.layers[-2].output])
        [conv_outputs, predictions] = get_output(x)

        conv_outputs = conv_outputs[0, :, :, :]

        cam = np.zeros(dtype=np.float32, shape=conv_outputs.shape[0:2])

        for index, weight in enumerate(class_weights[:, image_class]):
          if index<=2:
            cam += weight * conv_outputs[:, :, index]

        preds = predictions.argmax(axis=1)[0]
        files.append([image_path, np.max(predictions), preds])
        class_predicted = preds
        predictions = f'Class predicted: {class_predicted} | Real class: {image_class}'
        
        cam /= np.max(cam)
        cam = cv2.resize(cam, (img_size, img_size))
        heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam < 0.01)] = 0
        
        img = heatmap * 0.5 + image_loaded
        cv2.imwrite("heatmap.jpg", img)
        
        heatmap = mpimg.imread("heatmap.jpg")
        
        scaled_image = (((img - img.min())) / (img.max() - img.min())).astype(np.uint8)
        
        fig, ax = plt.subplots(1, 2, figsize=(10, 10))
        ax[0].imshow(image_loaded)

        ax[0].set_title('Original image')

        ax[1].imshow(heatmap)
        ax[1].set_title(predictions)

In [ ]:
def gradcam(vector, path):
  files = os.listdir(path)
  y = 0
  while y<len(files):
    img = files[y]
    img_path = path + img
    get_activation_map(img_path,vector,256)
    y+=1

print("COVID")
gradcam(vector=[1,0,0], path=covid_path)
print("None")
gradcam(vector=[0,1,0], path=none_path)
print("Pneumonia")
gradcam(vector=[0,0,1], path=pneumonia_path)